In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union, unary_union, transform
from datetime import datetime, timedelta
import datetime
import math
from scipy.ndimage.interpolation import shift
import shapely.wkt
from shapely.validation import explain_validity,make_valid
import xarray as xr
import pygeos as pg
import time
import seaborn as sns
from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines
from joblib import Parallel, delayed
import multiprocessing

In [110]:
#all fires
#fire_incidents = ['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH', 'WILLIAMS FLATS', 'SHADY','PEDRO MOUNTAIN', 'WALKER', '204 COW']

#2020 fires
#fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
fire_incidents = ['CAMERON PEAK']

path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
start_time=12
for jj in range(len(fire_incidents)):
    fire_daily = gpd.read_file('./fire_polygons/lake_VIIRS_daily_12Z_day_start.geojson')
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-08-15'),:]

    print(fire_daily)
    
    #merra
    #me = merra_timeseries(fire_daily, start_time)
    #print(me)
    #me.to_csv('./fire_features/'+fire_incidents[jj].lower().replace(' ', '_')+'_Daily_MERRA_Moving_Average_2_'+str(start_time)+'Z.csv') #daily averages

    #hrrr
    hrrr = hrrr_timeseries(fire_daily,start_time)
    print(hrrr)
    
    #rave
    #rave=rave_timeseries(fire_daily,start_time)

  12Z Start Day  Incident Number Fire Name     UTC Day   Lat Fire    Lon Fire  \
0    2020-08-12       11773470.0      LAKE  2020-08-13  34.678611 -118.451944   
1    2020-08-13       11773470.0      LAKE  2020-08-14  34.678611 -118.451944   
2    2020-08-14       11773470.0      LAKE  2020-08-15  34.678611 -118.451944   

   Number of NEW VIIRS points   NEW FRP  \
0                       132.0    782.77   
1                        25.0   1644.50   
2                        70.0  11606.51   

                                            geometry  fire area (ha)  
0  MULTIPOLYGON (((3641986.585 323212.339, 364197...     2056.378485  
1  MULTIPOLYGON (((3630083.512 322187.932, 363007...      196.114730  
2  MULTIPOLYGON (((3631093.725 323487.926, 363109...      796.663316  
0.6925725936889648
2020-08-12
DatetimeIndex(['2020-08-07 00:00:00', '2020-08-07 01:00:00',
               '2020-08-07 02:00:00', '2020-08-07 03:00:00',
               '2020-08-07 04:00:00', '2020-08-07 05:00:00',
     

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


[297.10569544]
[0.00374961]
[4.73482785]
[7.46602584]
[268.01640709]
[0.]
[3.39548825]
[38.16998767]
[0.00703334]
[38.16295433]
[27.94152084]
[4.73482785]
[7.46602584]
[268.01640709]
[0.98780489]
[5.12245831]
[29.09246185]
[0.95384999]
[297.10569544]
[0.00374961]
[4.73482785]
[0.98780489]
2020-08-13
DatetimeIndex(['2020-08-08 00:00:00', '2020-08-08 01:00:00',
               '2020-08-08 02:00:00', '2020-08-08 03:00:00',
               '2020-08-08 04:00:00', '2020-08-08 05:00:00',
               '2020-08-08 06:00:00', '2020-08-08 07:00:00',
               '2020-08-08 08:00:00', '2020-08-08 09:00:00',
               ...
               '2020-08-13 15:00:00', '2020-08-13 16:00:00',
               '2020-08-13 17:00:00', '2020-08-13 18:00:00',
               '2020-08-13 19:00:00', '2020-08-13 20:00:00',
               '2020-08-13 21:00:00', '2020-08-13 22:00:00',
               '2020-08-13 23:00:00', '2020-08-14 00:00:00'],
              dtype='datetime64[ns]', length=145, freq='H')


/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


[256.28388934]
[0.00515498]
[3.3619282]
[6.04931742]
[236.16936252]
[0.]
[4.92803647]
[33.84683848]
[0.00974692]
[33.83709156]
[12.08554703]
[3.3619282]
[6.04931742]
[236.16936252]
[0.84933891]
[3.46210564]
[20.11452243]
[0.80005856]
[256.28388934]
[0.00515498]
[3.3619282]
[0.84933891]
2020-08-14
DatetimeIndex(['2020-08-09 00:00:00', '2020-08-09 01:00:00',
               '2020-08-09 02:00:00', '2020-08-09 03:00:00',
               '2020-08-09 04:00:00', '2020-08-09 05:00:00',
               '2020-08-09 06:00:00', '2020-08-09 07:00:00',
               '2020-08-09 08:00:00', '2020-08-09 09:00:00',
               ...
               '2020-08-14 15:00:00', '2020-08-14 16:00:00',
               '2020-08-14 17:00:00', '2020-08-14 18:00:00',
               '2020-08-14 19:00:00', '2020-08-14 20:00:00',
               '2020-08-14 21:00:00', '2020-08-14 22:00:00',
               '2020-08-14 23:00:00', '2020-08-15 00:00:00'],
              dtype='datetime64[ns]', length=145, freq='H')


/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


[297.44087913]
[0.00672472]
[4.8173139]
[7.67276933]
[274.46825592]
[0.]
[2.77191869]
[41.89677744]
[0.01276115]
[41.88401628]
[23.02115491]
[4.8173139]
[7.67276933]
[274.46825592]
[0.98197069]
[5.18912904]
[22.97264655]
[0.95425145]
[297.44087913]
[0.00672472]
[4.8173139]
[0.98197069]
                   day       hd0w0       hd1w0       hd2w0       hd3w0  \
0  2020-08-12 12:00:00  180.695019  149.177556  140.936607  140.179795   
1  2020-08-13 12:00:00  113.757872  102.943724   88.987754   83.541909   
2  2020-08-14 12:00:00  201.768454  172.816177  165.752667  147.393035   

        hd4w0       hd5w0     temp_2m      q_2m  gust_sfc  ...     soilm  \
0  130.061124  114.790855  297.105695  0.003750  4.734828  ...  3.395488   
1   84.032850   78.010898  256.283889  0.005155  3.361928  ...  4.928036   
2  143.403234  141.807841  297.440879  0.006725  4.817314  ...  2.771919   

     esat_2m      e_2m     vpd_2m        hwp  veg_term  gust_max_term  \
0  38.169988  0.007033  38.162954  27.

## Dataset-Dependent Helper Functions

### MERRA

In [ ]:
def merra_timeseries(df,day_start_hour):
    df_merra = pd.DataFrame({'day': np.zeros(len(df)),'temp':np.zeros(len(df)), 'vpd':np.zeros(len(df)), 
                             'wind':np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))}) 
    
    #do the intersection, in parallel
    merra_intersections = Parallel(n_jobs=6)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'MERRA_GRID') 
                                 for ii in range(len(fire_daily)))
    fire_merra_intersection=gpd.GeoDataFrame(pd.concat(merra_intersections, ignore_index=True))
    fire_merra_intersection.set_geometry(col='geometry')
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_merra_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        #get the time
        df_sub = fire_merra_intersection.iloc[np.where(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
        intersection_sub = df_sub.to_xarray() #polygon and weights for today

        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+np.timedelta64(1,'D'))
        files_back = make_merra_file_namelist(times_back)
        
        #load in all the merra files associated with this lookback window
        dat_merra = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    
        #add the derived data (svp, vp, vpd)
        dat_merra=dat_merra.assign(ESAT=sat_vap_press(dat_merra.TLML))
        dat_merra=dat_merra.assign(E=vap_press(dat_merra.QLML, dat_merra.TLML))
        dat_merra=dat_merra.assign(VPD=dat_merra.ESAT-dat_merra.E)
        
        merra_daily_mean = dat_merra.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        merra_daily_mean_region = merra_daily_mean.sel(lat = np.unique(intersection_sub['lat'].values),
                                  lon = np.unique(intersection_sub['lon'].values)) #get the location of the overlaps
        
        hd0 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((merra_daily_mean_region['SPEEDLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((merra_daily_mean_region['TLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_merra.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',t,hd0,w,hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        dat_merra.close()
        count =count+1
    return df_merra
    

In [ ]:
def make_merra_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.FULLDATE.nc4'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
        if (time[jj].strftime('%Y%m')=='202009'):
            base_filename_list[jj] = base_filename_list[jj].replace('400','401')
    return base_filename_list

### HRRR

In [109]:
def hrrr_timeseries(df,day_start_hour):  
    df_hrrr_derived = pd.DataFrame({'day': np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    
    df_hrrr_raw = pd.DataFrame({'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'weasd':np.zeros(len(df)),'soilm':np.zeros(len(df)), 'esat_2m':np.zeros(len(df)),
                             'e_2m': np.zeros(len(df)),'vpd_2m':np.zeros(len(df)), 'hwp':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'veg_term':np.zeros(len(df)),'gust_max_term':np.zeros(len(df)), 'dd_term':np.zeros(len(df)),
                             'mois_term': np.zeros(len(df)),'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'snowc_term':np.zeros(len(df))})
    
    varis = ['temp_2m', 'q_2m','gust_sfc','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m',
             'vpd_2m', 'hwp','gust_sfc','veggie', 'dewpt','veg_term','gust_max_term', 'dd_term',
            'mois_term','temp_2m', 'q_2m','gust_sfc','snowc_term']
    
    #do the intersection, in parallel
    tic=time.time()
    hrrr_intersections = Parallel(n_jobs=6)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID') 
                                 for ii in range(len(fire_daily)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    toc = time.time()
    print(toc-tic)
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back = make_hrrr_file_namelist(times_back)
        
        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back})

        #add the derived data (svp, vp, vpd)
        dat_hrrr=dat_hrrr.assign(esat_2m=sat_vap_press(dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(e_2m=vap_press(dat_hrrr.q_2m, dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(VPD=dat_hrrr.esat_2m-dat_hrrr.e_2m)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps

        hd0 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((hrrr_daily_mean_region['gust_sfc'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((hrrr_daily_mean_region['temp_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_hrrr_derived.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        
        for var in varis:
            df_hrrr_raw[var].iloc[count] = np.nansum(intersection_sub['weights'].values*hrrr_daily_mean_region[var].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values, axis=(1,2))
        
        df_hrrr = pd.concat([df_hrrr_derived,df_hrrr_raw],axis=1)
        dat_hrrr.close()
        count =count+1
    return df_hrrr

In [51]:
def make_hrrr_file_namelist(time):
    base_filename = '/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YYYYMMDDHH',time[jj].strftime('%Y%m%d%H'))
    return base_filename_list

In [41]:
times_back = pd.date_range(start=np.datetime64('2020-08-01'), end=np.datetime64('2020-08-02'),freq='H')
print(times_back)

listi = make_hrrr_file_namelist(times_back)
print(listi)

DatetimeIndex(['2020-08-01 00:00:00', '2020-08-01 01:00:00',
               '2020-08-01 02:00:00', '2020-08-01 03:00:00',
               '2020-08-01 04:00:00', '2020-08-01 05:00:00',
               '2020-08-01 06:00:00', '2020-08-01 07:00:00',
               '2020-08-01 08:00:00', '2020-08-01 09:00:00',
               '2020-08-01 10:00:00', '2020-08-01 11:00:00',
               '2020-08-01 12:00:00', '2020-08-01 13:00:00',
               '2020-08-01 14:00:00', '2020-08-01 15:00:00',
               '2020-08-01 16:00:00', '2020-08-01 17:00:00',
               '2020-08-01 18:00:00', '2020-08-01 19:00:00',
               '2020-08-01 20:00:00', '2020-08-01 21:00:00',
               '2020-08-01 22:00:00', '2020-08-01 23:00:00',
               '2020-08-02 00:00:00'],
              dtype='datetime64[ns]', freq='H')
['/data2/lthapa/ML_py/pygraf/Processed_HRRR_2020080100.nc4'
 '/data2/lthapa/ML_py/pygraf/Processed_HRRR_2020080101.nc4'
 '/data2/lthapa/ML_py/pygraf/Processed_HRRR_2020080102.nc4'
 

### RAVE

In [19]:
def rave_timeseries(df, day_start_hour):
    varis = ['Mean_FRP', 'FRE', 'CO2', 'CO', 'SO2', 'OC', 'BC', 'PM2.5', 'NOx', 'NH3'] #don't need 'area', it's the area of each cell


    #do the intersection, in parallel
    rave_intersections = Parallel(n_jobs=6)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'RAVE_GRID') 
                                 for ii in range(len(fire_daily)))
    fire_rave_intersection=gpd.GeoDataFrame(pd.concat(rave_intersections, ignore_index=True))
    fire_rave_intersection.set_geometry(col='geometry')
    
    
    fire_rave_intersection = fire_rave_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    print(fire_rave_intersection)
    fire_rave_intersection = fire_rave_intersection.to_xarray()
    """
    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    rave_filenames = make_rave_file_namelist(times)
    dat_rave = xr.open_mfdataset(rave_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    dat_rave = dat_rave.assign_coords({'Time': dat_rave.time}) #assign coords so we can resample along time
    dat_rave = dat_rave.resample(Time='24H',base=day_start_hour).sum(dim='Time') #take the daily sum

    
    #select the locations and times we want
    dat_rave_sub = dat_rave.isel(yFRP = fire_rave_intersection_xr['row'].values.astype(int), 
                    xFRP = fire_rave_intersection_xr['col'].values.astype(int)).sel(
                    Time = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                         'T'+str(day_start_hour)+':00:00'))
    ndays = len(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_rave = pd.DataFrame({'day':np.zeros(ndays),'Mean_FRP':np.zeros(ndays),\
                          'FRE':np.zeros(ndays),  'CO2':np.zeros(ndays),  'CO':np.zeros(ndays),\
                          'SO2':np.zeros(ndays),  'OC':np.zeros(ndays),  'BC':np.zeros(ndays),\
                          'PM2.5':np.zeros(ndays),  'NOx':np.zeros(ndays),  'NH3':np.zeros(ndays)})

    df_rave['day'].iloc[:] = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                         'T'+str(day_start_hour)+':00:00')
    for var in varis:
        df_rave[var] = np.nansum(fire_rave_intersection_xr['weights'].values*dat_rave_sub[var].values, axis=(1,2))
    
    return df_rave
    """

## Dataset-Independent Helper Functions

In [3]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_one_gridcell(LAT_COR, LON_COR, LAT_CTR, LON_CTR, ii,jj):
    #print(ii,jj,count)
    #print(LAT_CTR[ii,jj], LON_CTR[ii,jj]) #ctr
    sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
    se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
    nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
    ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
            
    poly_cell = Polygon([sw,nw,ne,se])
    
    return LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell
    

In [4]:
#poly is the polygon for one timestep (in lcc)
#grid is an xarray of a model grid from the nc file
#grid_names is a string array [0:'lat_center_name',1:'lon_center_name',2:'lat_corner_name',3:'lon_corner_name']

def calculate_intersection(poly,dataset_name):
    #load in the merra grid
    grid = xr.open_dataset(dataset_name+'.nc')

    #for each fire_daily polygon

    #get the bounds
    poly_latlon =poly.to_crs(epsg=4326)
    bounds = poly_latlon.bounds
    print(bounds)
    
    #first check for rows and cols
    [rows,cols] = np.where((grid.LAT_CTR.values>bounds['miny'].values)&
                    (grid.LAT_CTR.values<bounds['maxy'].values)&
                    (grid.LON_CTR.values>bounds['minx'].values)&
                    (grid.LON_CTR.values<bounds['maxx'].values))

    if rows.size==0: #if the whole polygon is inside one grid cell, find that grid cell
        print('empty!')
        lat_middle = (bounds['miny'].values+bounds['maxy'].values)/2
        lon_middle = (bounds['minx'].values+bounds['maxx'].values)/2

        distance = np.sqrt((grid.LAT_CTR-lat_middle)**2+(grid.LON_CTR-lon_middle)**2)
        row_min_location,col_min_location = np.where(distance ==np.min(distance))
        rows = np.append(rows,row_min_location)
        cols = np.append(cols,col_min_location)
    
    print(rows,cols)
    
    
    #make a geodataframe (in paralell of the rows and cols)
    tic = time.time()
    results = Parallel(n_jobs=6)(delayed(build_one_gridcell)
                                 (grid['LAT_COR'].values, grid['LON_COR'].values,
                                  grid['LAT_CTR'].values, grid['LON_CTR'].values,i,j) 
                                 for i in rows for j in cols)
    toc = time.time()
    print(toc-tic)
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:4326') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)
    #print(df_grid)
    
    #intersect the polygon with the grid subset
    intersection = gpd.overlay(poly, df_grid, how='intersection',keep_geom_type=False)
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    
    return intersection
    

# Testing

In [ ]:
tic = time.time()
merra_intersections = Parallel(n_jobs=6)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'MERRA_GRID') 
                                 for ii in range(len(fire_daily)))
toc=time.time()

merra_intersections_gdf=gpd.GeoDataFrame(pd.concat(merra_intersections, ignore_index=True))
merra_intersections_gdf.set_geometry(col='geometry')
print(merra_intersections_gdf)
print(toc-tic)
tic = time.time()
hrrr_intersections = Parallel(n_jobs=6)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID') 
                                 for ii in range(len(fire_daily)))
toc=time.time()

hrrr_intersections_gdf=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
hrrr_intersections_gdf.set_geometry(col='geometry')
print(hrrr_intersections_gdf)
print(toc-tic)
